# BCC Puzzle for Today Nr. 78

This program solves the [BBC Puzzle for Today Nr. 78](http://www.bbc.co.uk/programmes/articles/4GrCLXtfVmYjhSNCvcg6lSp/puzzle-for-today)

The problem is to find

    1.) The number of Palindrome Times in 24hrs.
    2.) The minimum time between any Palindrome times

# Datastructure

A first analysis of the problem allows us to reason that

    1.) A palindrome time is a time such that
    
        1.1) It has the form AB:CC:BA where A,B,C are digits
        
        1.2) AB must be between 00 and 23, CC must be between 
             00 and 60, and BA must be between 00 and 60
             
We need to represent a triplet of digits, so a list of integers seems like a good choice.

So we will represent a palindrome time as a triplet `(A,B,C)` where `A`,`B`,`C` are digits between `0` to `9`.

In [1]:
palindrome_time = (0,0,0)

So we will first define a function to create the complete time description from the palindrome time specification.

Since the next checks require us to use the *hours* *(A * 10 + B)*, we will return the number of hours, minutes, and seconds.

In [2]:
def createTime( ptime ): # (A,B,C)
    times = None
    a,b,c = ptime
    if ( ( len(ptime) == 3 ) and (a >= 0 ) and (a <= 9 )
                             and (b >= 0 ) and (b <= 9 )
                             and (c >= 0 ) and (c <= 9 ) ):
        if ( a > 0 ):
            time1 = ( a*10+b, # AB 
                  c*10+c, # CC
                  b*10+a) # BA
        else:
            time1 = ( 99, 99, 99 )
        time2 = ( a,            # 0A
                  b * 10 + c,   # BC
                  b*10+a )       # BA
    return (time1, time2)

We can then test our `createTime` function with some simple calls.

In [3]:
print(createTime((0,0,0)))
print(createTime((1,2,5)))
print(createTime((9,2,3)))

((99, 99, 99), (0, 0, 0))
((12, 55, 21), (1, 25, 21))
((92, 33, 29), (9, 23, 29))


In [4]:
def checkTime( time ):
    hrs, mins, secs = time
    result = False
    if ( ( hrs >= 0 ) and ( hrs <= 23 ) 
        and ( mins >= 0 ) and ( mins <= 59 )
        and ( secs >= 0 ) and ( secs <= 59 ) ):
        result = True
    return result

We also create a function `printTime`, which prints the time in 24 hr format.

In [5]:
def printTime( time ):
    hrs, mins, secs = time
    s = "{0:2d}:{1:02d}:{2:02d}".format(hrs, mins, secs)
    return s

Now we can check if the created times are legal or not.

In [6]:
for t in [ ( 0,0,0), (1,2,5), (9,2,3) ]:
    t1,t2 = createTime(t)
    print('Checking ptime', t, printTime(t1), checkTime(t1) )
    print('Checking ptime', t, printTime(t2), checkTime(t2) )


Checking ptime (0, 0, 0) 99:99:99 False
Checking ptime (0, 0, 0)  0:00:00 True
Checking ptime (1, 2, 5) 12:55:21 True
Checking ptime (1, 2, 5)  1:25:21 True
Checking ptime (9, 2, 3) 92:33:29 False
Checking ptime (9, 2, 3)  9:23:29 True


# Generate all possible palindrome times

Let us generate all possible palindrome times by enumerating all possible A, B, C.

In [7]:
ptimes = []
for a in range(0,10):
    for b in range( 0, 10):
        for c in range( 0, 10):
            t1,t2 = createTime( (a,b,c) )
            if (checkTime( t1 ) and (printTime(t1) not in ptimes) ):
                ptimes = ptimes + [printTime(t1)]
            if (checkTime( t2 ) and (printTime(t2) not in ptimes) ):
                ptimes = ptimes + [printTime(t2)]
print('Number of palindrome times', len(ptimes))

Number of palindrome times 660


Now let us find the closest time between any two palindrome times.

First create a function that calculates the minimum difference between times.

In [8]:
def findMinimumDifference( ptimes ):
    mint = 25 * 3600
    ctime1 = None
    ctime2 = None

    for i in range(len(ptimes)):
        h1,m1,s1 = ptimes[i]
        t1 = h1 * 3600 + m1 * 60 + s1
        for j in range( i+1, len(ptimes) ):
            h2,m2,s2 = ptimes[j]
            t2 = h2 * 3600 + m2 * 60 + s2
            difft = t1 - t2
            if ( difft < 0 ):
                difft = -difft
            if ( difft > 12 * 3600 ):
                difft = 24 * 3600 - difft
            if ( difft < mint ):
                mint = difft
                ctime1 = ptimes[i]
                ctime2 = ptimes[j]
    return ( mint, ctime1, ctime2 )


In [9]:
ptimes = []
for a in range(0,10):
    for b in range( 0, 10):
        for c in range( 0, 10):
            t1,t2 = createTime( (a,b,c) )
            if (checkTime( t1 ) and (t1 not in ptimes) ):
                ptimes = ptimes + [t1]
            if ( checkTime( t2 ) and ( t2 not in ptimes) ):
                ptimes = ptimes + [t2]

mint, ctime1, ctime2 = findMinimumDifference( ptimes )
print('Min. time difference', mint, 'time1', printTime(ctime1), 'time2', printTime(ctime2 ))



Min. time difference 2 time1 10:00:01 time2  9:59:59


# Extension: Palindrome times made up of unique digits

Let us now only consider palindrome times that are made up of unique digits a,b,c.

In [10]:
ptimes = []
for a in range(0,10):
    for b in range( 0, 10):
        for c in range( 0, 10):
            if ( a != b ) and ( a != c ) and ( b != c ):
                t1,t2 = createTime( (a,b,c) )
                if (checkTime( t1 ) and (t1 not in ptimes) ):
                    ptimes = ptimes + [t1]
                if ( checkTime( t2 ) and ( t2 not in ptimes) ):
                    ptimes = ptimes + [t2]

mint, ctime1, ctime2 = findMinimumDifference( ptimes )
print('Number of palindrome times with unique digits', len(ptimes) )
print('Min. time difference', mint, 'time1', printTime(ctime1), 'time2', printTime(ctime2 ))


Number of palindrome times with unique digits 464
Min. time difference 60 time1  0:12:10 time2  0:13:10
